**Práctica 4: Visualización de información con plotly**

Ahora vamos a dibujar todo lo que en la práctica pasada parecía que habría que graficarlo.
Para que las gráficas sean interactuables con plotly en jupyter, 
primero hay que extraer los datos para graficar *sin información personal* de los estudiantes 
en archivos CSV aparte que se pueda compartir en línea.

Vamos a quedarnos únicamente con las columnas que se analizaron en la práctica anterior 
y únicamente con los alumnos de Moisés y Elisa ya que no hemos incorporado las calificaciones de los demás aún.

In [ ]:
import pandas as pd
from numpy import NaN
d = pd.read_csv("casos.csv")
d['CF1op'] = d['1ra']
d.CF1op = [int(v) if str(v).isdigit() else NaN for v in d.CF1op]
d['CF2op'] = d['2da']
d.CF2op = [int(v) if str(v).isdigit() else NaN for v in d.CF2op]
d.ingreso = ["AD" + v[-2:] if "dic" in v else "EJ" + v[-2:] if "nero" in v else NaN for v in [str(v) for v in d.ingreso]]
d.ingreso = d.ingreso.replace('ADre', NaN)
d['inicio'] = ['enero' if 'EJ' in v else 'agosto' if 'AD' else NaN for v in [str(v) for v in d.ingreso]]
hrs = {"No tengo un trabajo": 0, \
       "Menos de 10 horas": 5, \
       "Entre 10 y 20 horas": 15, \
       "Entre 20 y 40 horas": 30, \
       "Más de 40 horas": 50}
d['hrsNum'] = [hrs.get(v, NaN) for v in d.hrsTrabajo]
d['sabePromedio'] =  [s != 'nan' for s in [str(v) for v in d.prom]]
d['sabeCreditos'] = d.creditos == 'Tres.'
d['sabeHoras']  = d['corr'] == '30'
d['sabeAmbos'] = d.sabeCreditos & d.sabeHoras
d['noSabeCreditos'] = d.creditos != "Tres."
d['noSabeHoras']  = d['corr'] != '30'
d['noSabeNinguno'] = d.noSabeCreditos & d.noSabeHoras
viejos = ["segunda", "70-79", "80-89", "90-100", "2da", "3ra", \
          "No creo aprobar en este semestre", "Estimo aprobar en segunda oportunidad", "Creo que aprobaré en segunda oportunidad"]
nuevos = [50, 75, 85, 95, 50, 20, 20, 50, 50] # el precio de la inconsistencia es la redundancia
nn = ['I', 'M', 'F']
for s in ["_ini", "_mcu", ""]:
    d['e' + nn.pop(0)] = d["califEsp" + s].replace(viejos, nuevos) 
d['fueAses'] = ["NA" if v is NaN else False if str(v) == "No" else True for v in d.asesorias]
d['cuantasTemas'] = [0 if len(t) < 1 else t.count(';') + 1 for t in [str(v) if v is not NaN else '' for v in d.temasGral]]
d['formasApoyo'] = [0 if len(t) < 1 else t.count(';') + 1 for t in [str(v) if v is not NaN else '' for v in d.apoyo]]
d['cuantosMedios'] = [0 if len(t) < 1 else t.count(';') + 1 for t in [str(v) if v is not NaN else '' for v in d.medios]]
d['analizable'] = [p in ['elisa', 'moi'] for p in d.profe]
pedazo = d.query('analizable')
keep = ['profe', 'grupo', 'sem', 'PE', 'CF1op', 'CF2op', 'ingreso', 'inicio', 'hrsNum', 'sabePromedio', \
        'sabeCreditos', 'sabeHoras', 'sabeAmbos', 'noSabeCreditos', 'noSabeHoras', 'noSabeNinguno', \
        'eI', 'eM',  'eF', 'fueAses', 'temas', 'cuantasTemas', 'formasApoyo', 'cuantosMedios', \
        'hrsEstudio_ini', 'hrsEstudio_mcu', 'hrsEstudio']
extracto = pd.DataFrame(pedazo, columns = keep)
extracto.to_csv("graficar.csv")

In [ ]:
python3 extract.py
wc -l graficar.csv 
     580 graficar.csv
wc -l casos.csv 
    1023 casos.csv

Ahora, ese archivo graficar.csv se puede poner en la web, 
lo que nos permite graficarlo con plotly aquí mismo en el notebook de jupyter.

Lo que sí hay que decirle es que no se preocupe por verificar el certificado SSL del HTTPS.

In [15]:
import plotly.plotly as py
import plotly.graph_objs as go
import pandas as pd
import ssl

if getattr(ssl, '_create_unverified_context', None):
    ssl._create_default_https_context = ssl._create_unverified_context
d = pd.read_csv("https://elisa.dyndns-web.com/teaching/comp/datasci/graficar.csv")
datos = []
for sem in d.ingreso.unique():
    if sem != NaN:
        datos.append(go.Box(y = d.loc[d['ingreso'] == sem].CF1op, name = sem))
g = py.iplot(datos, filename='jupyter-semestre_primera')
print(g.embed_code)

<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="https://plot.ly/~satuelisa/6.embed" height="525px" width="100%"></iframe>


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="https://plot.ly/~satuelisa/6.embed" height="525px" width="100%"></iframe>

El resultado, igual como todos los demás, están además en __[una página aparte](https://elisa.dyndns-web.com/teaching/comp/datasci/p4.html#f1)__ para interactuar con ellos directamente.

El orden de los semestres en el eje horizontal no convence.
Tiene que existir una forma de reordenarlos.

El chiste es reordenar el data frame para que los semestres aparezcan en el orden lógico: 
primero por año y luego que EJ venga antes de AD.

Mejor primero chequemos cuál es el año más antiguo y el más reciente en aparecer:

In [3]:
import pandas as pd
from numpy import NaN
import ssl

if getattr(ssl, '_create_unverified_context', None):
    ssl._create_default_https_context = ssl._create_unverified_context
d = pd.read_csv("https://elisa.dyndns-web.com/teaching/comp/datasci/graficar.csv")
tmp = [NaN if v == 'nan' else v[2:] for v in [str(v) for v in d.ingreso]]
d['aIng'] = [NaN if v == 'nan' else '20' + v if int(v) < 20 else '19' + v for v in [str(v) for v in tmp]]
print(d.aIng.unique())

['2012' nan '2014' '2017' '2016' '2015' '2018' '2013' '1995']


OK, tenemos los años. Ahora los semestres. 
Pongamos .1 para EJ y .2 para AD para tener una codificación numérica.
Así será fácil de ordenar.

In [5]:
import pandas as pd
from numpy import NaN
import ssl

if getattr(ssl, '_create_unverified_context', None):
    ssl._create_default_https_context = ssl._create_unverified_context
d = pd.read_csv("https://elisa.dyndns-web.com/teaching/comp/datasci/graficar.csv")
tmp = [NaN if v == 'nan' else v[2:] for v in [str(v) for v in d.ingreso]]
d['aIng'] = [NaN if v == 'nan' else '20' + v if int(v) < 20 else '19' + v for v in [str(v) for v in tmp]]
d['sIng'] = [NaN if v == 'nan' else v[:2] for v in [str(v) for v in d.ingreso]]
d['saIng'] = [NaN if x is NaN else int(x) + (0.1 if y == "AD" else 0.2) for x, y in zip(d.aIng, d.sIng)]
print(d.saIng.unique())

[ 2012.1     nan  2014.1  2017.2  2014.2  2016.2  2016.1  2017.1  2015.2
  2018.2  2015.1  2013.2  2013.1  1995.2]


In [ ]:
import pandas as pd
from numpy import NaN
import ssl
import plotly.plotly as py
import plotly.graph_objs as go

if getattr(ssl, '_create_unverified_context', None):
    ssl._create_default_https_context = ssl._create_unverified_context
d = pd.read_csv("https://elisa.dyndns-web.com/teaching/comp/datasci/graficar.csv")
tmp = [NaN if v == 'nan' else v[2:] for v in [str(v) for v in d.ingreso]]
d['aIng'] = [NaN if v == 'nan' else '20' + v if int(v) < 20 else '19' + v for v in [str(v) for v in tmp]]
d['sIng'] = [NaN if v == 'nan' else v[:2] for v in [str(v) for v in d.ingreso]]
d['saIng'] = [NaN if x is NaN else int(x) + (0.1 if y == "EJ" else 0.2) for x, y in zip(d.aIng, d.sIng)]
ordenado = d.sort_values(by = "saIng")
datos = []
for sem in ordenado.ingreso.unique():
    if sem != NaN:
        datos.append(go.Box(y = d.loc[d['ingreso'] == sem].CF1op, name = sem))
g = py.iplot(datos, filename='jupyter-semestre_primera_v2')
print(g.embed_code)

<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="https://plot.ly/~satuelisa/8.embed" height="525px" width="100%"></iframe>

__[El resultado](https://elisa.dyndns-web.com/teaching/comp/datasci/p4.html#f2)__ ya es lo que buscaba.

Por lo menos con estos alumnos, la leyenda urbana de que los que inicien en enero estén peores no aplica. 
No se nota ningún patrón sistemático alternación en las calificaciones de primera oportunidad.

Hagamos algo parecido por programa educativo, 
poniendo boxplots de un color para 1ra op. y de otro color para 2da op.

In [7]:
import pandas as pd
from numpy import NaN
import ssl
import plotly.plotly as py
import plotly.graph_objs as go
from random import randint

if getattr(ssl, '_create_unverified_context', None):
    ssl._create_default_https_context = ssl._create_unverified_context
d = pd.read_csv("https://elisa.dyndns-web.com/teaching/comp/datasci/graficar.csv")

datos = []
rojo = 69
for car in d.PE.unique():
    if car != NaN:
        pedazo = d.loc[d['PE'] == car]
        verde = {"color": 'rgb({:d}, {:d}, {:d})'.format(rojo, randint(170, 255), randint(0, 50))}
        azul = {"color": 'rgb({:d}, {:d}, {:d})'.format(rojo, randint(0, 50), randint(170, 255))}
        rojo = (rojo + 41) % 255
        if pedazo.CF1op.count() > 0:
            datos.append(go.Box(y = d.loc[d['PE'] == car].CF1op, name = str(car) + " 1ra", marker = verde))
        if pedazo.CF2op.count() > 0:
            datos.append(go.Box(y = d.loc[d['PE'] == car].CF2op, name = str(car) + " 2da", marker = azul))
g = py.iplot(datos, filename='jupyter-carrera')
print(g.embed_code)

<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="https://plot.ly/~satuelisa/10.embed" height="525px" width="100%"></iframe>


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="https://plot.ly/~satuelisa/10.embed" height="525px" width="100%"></iframe>

En el __[resultado](https://elisa.dyndns-web.com/teaching/comp/datasci/p4.html#f3)__ es un poco difícil comprar las carreras porque la primera y la segunda oportunidad tienen una variación muy distinta.

Hagamos otra versión, ordenado primero por oportunidad y luego por carrera.

In [ ]:
import pandas as pd
from numpy import NaN
import ssl
import plotly.plotly as py
import plotly.graph_objs as go
from random import randint

if getattr(ssl, '_create_unverified_context', None):
    ssl._create_default_https_context = ssl._create_unverified_context
d = pd.read_csv("https://elisa.dyndns-web.com/teaching/comp/datasci/graficar.csv")

datos = []
colores = dict()
rojo = 69
for car in d.PE.unique():
    if car != NaN:
        pedazo = d.loc[d['PE'] == car]
        verde = {"color": 'rgb({:d}, {:d}, {:d})'.format(rojo, randint(170, 255), randint(0, 50))}
        azul = {"color": 'rgb({:d}, {:d}, {:d})'.format(rojo, randint(0, 50), randint(170, 255))}
        rojo = (rojo + 41) % 255
        if pedazo.CF1op.count() > 0:
            datos.append(go.Box(y = d.loc[d['PE'] == car].CF1op, name = str(car) + " 1ra", marker = verde))
            colores[car] = (verde, azul)
for car in d.PE.unique():
    if car != NaN:
        if car not in colores:
            colores[car] = ({"color": 'rgb({:d}, {:d}, {:d})'.format(rojo, randint(170, 255), randint(0, 50))}, \
                            {"color": 'rgb({:d}, {:d}, {:d})'.format(rojo, randint(0, 50), randint(170, 255))})
            rojo = (rojo + 41) % 255
        (verde, azul) = colores[car]  
        pedazo = d.loc[d['PE'] == car]
        if pedazo.CF2op.count() > 0:
            datos.append(go.Box(y = d.loc[d['PE'] == car].CF2op, name = str(car) + " 2da", marker = azul))
g = py.iplot(datos, filename='jupyter-carrera_v2')
print(g.embed_code)

<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="https://plot.ly/~satuelisa/12.embed" height="525px" width="100%"></iframe>

Este __[resultado](https://elisa.dyndns-web.com/teaching/comp/datasci/p4.html#f4)__ ya es más fácil de interpretar.

Al parecer las segundas de Moisés para los de materiales son de tipo 70 o nada. 

Ibamos a graficar también lo de por cuántas horas planearon estudiar y cuántas horas dijeron que estudiaron en la primera y en la segunda mitad del semestre.

In [10]:
import pandas as pd
from numpy import NaN
import ssl
import plotly.plotly as py
import plotly.graph_objs as go

if getattr(ssl, '_create_unverified_context', None):
    ssl._create_default_https_context = ssl._create_unverified_context
d = pd.read_csv("https://elisa.dyndns-web.com/teaching/comp/datasci/graficar.csv")

datos = []
for hrs in d.hrsEstudio_ini.unique():
    if hrs != NaN:
        pedazo = d.loc[d['hrsEstudio_ini'] == hrs]
        if pedazo.CF1op.count() > 0:
            datos.append(go.Box(y = d.loc[d['hrsEstudio_ini'] == hrs].CF1op, name = hrs))
l = go.Layout(
    title='Encuesta al inicio del semestre',
    xaxis=dict(
        title='Horas planeadas de estudio por semana',
        titlefont=dict(
            size=14,
            color='#000000'
        )
    ),
    yaxis=dict(
        title='Calificación en primera oportunidad',
        titlefont=dict(
            size=14,
            color='#000000'
        )
    )
)
f = go.Figure(data = datos, layout = l)
g = py.iplot(f, filename='jupyter-hrs_ini')
print(g.embed_code)
datos = []
for hrs in d.hrsEstudio_mcu.unique():
    if hrs != NaN:
        pedazo = d.loc[d['hrsEstudio_mcu'] == hrs]
        if pedazo.CF1op.count() > 0:
            datos.append(go.Box(y = d.loc[d['hrsEstudio_mcu'] == hrs].CF1op, name = hrs))
l = go.Layout(
    title='Encuesta después del examen de medio curso',
    xaxis=dict(
        title='Horas reportadas de estudio por semana',
        titlefont=dict(
            size=14,
            color='#000000'
        )
    ),
    yaxis=dict(
        title='Calificación en primera oportunidad',
        titlefont=dict(
            size=14,
            color='#000000'
        )
    )
)
f = go.Figure(data = datos, layout = l)
g = py.iplot(f, filename='jupyter-hrs_mcu')
print(g.embed_code)
datos = []
for hrs in d.hrsEstudio.unique():
    if hrs != NaN:
        pedazo = d.loc[d['hrsEstudio'] == hrs]
        if pedazo.CF1op.count() > 0:
            datos.append(go.Box(y = d.loc[d['hrsEstudio'] == hrs].CF1op, name = hrs))
l = go.Layout(
    title='Encuesta antes del examen de ordinario',
    xaxis=dict(
        title='Horas reportadas de estudio por semana en la segunda mitad',
        titlefont=dict(
            size=14,
            color='#000000'
        )
    ),
    yaxis=dict(
        title='Calificación en primera oportunidad',
        titlefont=dict(
            size=14,
            color='#000000'
        )
    )
)
f = go.Figure(data = datos, layout = l)
g = py.iplot(f, filename='jupyter-hrs_ord')
print(g.embed_code)

<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="https://plot.ly/~satuelisa/14.embed" height="525px" width="100%"></iframe>
<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="https://plot.ly/~satuelisa/16.embed" height="525px" width="100%"></iframe>
<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="https://plot.ly/~satuelisa/18.embed" height="525px" width="100%"></iframe>


<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="https://plot.ly/~satuelisa/14.embed" height="525px" width="100%"></iframe>
<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="https://plot.ly/~satuelisa/16.embed" height="525px" width="100%"></iframe>
<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="https://plot.ly/~satuelisa/18.embed" height="525px" width="100%"></iframe>

En las figuras __[5](https://elisa.dyndns-web.com/teaching/comp/datasci/p4.html#f5)__, __[6](https://elisa.dyndns-web.com/teaching/comp/datasci/p4.html#f6)__ y __[7](https://elisa.dyndns-web.com/teaching/comp/datasci/p4.html#f7)__ 
el orden y texto de las opciones varía de una figura a otra y por ende también el color,
lo que dificulta la comparación. Además se empalman las etiquetas.
Mejor ponerles etiquetas consistentes, siempre en el mismo orden y con el mismo color.

In [16]:
import plotly.plotly as py
import plotly.graph_objs as go
import pandas as pd
from numpy import NaN
import ssl

if getattr(ssl, '_create_unverified_context', None):
    ssl._create_default_https_context = ssl._create_unverified_context
d = pd.read_csv("https://elisa.dyndns-web.com/teaching/comp/datasci/graficar.csv")
d.hrsEstudio_ini = d.hrsEstudio_ini.replace("Menos de una hora", "< 1 h")
d.hrsEstudio_ini = d.hrsEstudio_ini.replace("Entre 1 y 2 horas", "1-2 h")
d.hrsEstudio_ini = d.hrsEstudio_ini.replace("Entre 2 y 3 horas", "2-3 h")
d.hrsEstudio_ini = d.hrsEstudio_ini.replace("Entre 3 y 5 horas", "3-5 h")
d.hrsEstudio_ini = d.hrsEstudio_ini.replace("Más de 5 horas", "> 5 h")
print(d.hrsEstudio_ini.unique())
d.hrsEstudio_mcu = d.hrsEstudio_mcu.replace("Una hora o menos", "< 1 h")
d.hrsEstudio_mcu = d.hrsEstudio_mcu.replace("Menos de una hora", "< 1 h") # hubo dos formas decir esencialmente lo mismo                                                                                           
d.hrsEstudio_mcu = d.hrsEstudio_mcu.replace("Entre una y dos horas", "1-2 h")
d.hrsEstudio_mcu = d.hrsEstudio_mcu.replace("Entre dos y tres horas", "2-3 h")
d.hrsEstudio_mcu = d.hrsEstudio_mcu.replace("Entre tres y cinco horas", "3-5 h")
d.hrsEstudio_mcu = d.hrsEstudio_mcu.replace("Más de cinco horas", "> 5 h")
print(d.hrsEstudio_mcu.unique()) # que nada siga siendo nada                                                                                                                                                       
d.hrsEstudio = d.hrsEstudio.replace("Menos de una hora", "< 1 h")
d.hrsEstudio = d.hrsEstudio.replace("Entre una y dos horas", "1-2 h")
d.hrsEstudio = d.hrsEstudio.replace("Entre dos y tres horas", "2-3 h")
d.hrsEstudio = d.hrsEstudio.replace("Entre tres y cinco horas", "3-5 h")
d.hrsEstudio = d.hrsEstudio.replace("Más de cinco horas", "> 5 h")
print(d.hrsEstudio.unique())
orden = ["Nada", "< 1 h", "1-2 h", "2-3 h", "3-5 h", "> 5 h"]
colores = dict()
r = 29
g = 41
b = 67
for o in orden:
    colores[o] = {"color": 'rgb({:d}, {:d}, {:d})'.format(r, g, b)}
    r = (r + 29) % 255
    g = (g + 41) % 255
    b = (b + 67) % 255

datos = []
for hrs in orden:
    if hrs != NaN:
        pedazo = d.loc[d['hrsEstudio_ini'] == hrs]
        if pedazo.CF1op.count() > 0:
            datos.append(go.Box(y = d.loc[d['hrsEstudio_ini'] == hrs].CF1op, name = hrs, marker = colores[hrs]))
l = go.Layout(
    title='Encuesta al inicio del semestre',
    xaxis=dict(
        title='Horas planeadas de estudio por semana',
        titlefont=dict(
            size=14,
            color='#000000'
        )
    ),
    yaxis=dict(
        title='Calificación en primera oportunidad',
        titlefont=dict(
            size=14,
            color='#000000'
        )
    )
)
f = go.Figure(data = datos, layout = l)
g = py.iplot(f, filename='jupyter-hrs_ini_v2')
print(g.embed_code)
datos = []
for hrs in orden:
    if hrs != NaN:
        pedazo = d.loc[d['hrsEstudio_mcu'] == hrs]
        if pedazo.CF1op.count() > 0:
            datos.append(go.Box(y = d.loc[d['hrsEstudio_mcu'] == hrs].CF1op, name = hrs, marker = colores[hrs]))
l = go.Layout(
    title='Encuesta después del examen de medio curso',
    xaxis=dict(
        title='Horas reportadas de estudio por semana',
        titlefont=dict(
            size=14,
            color='#000000'
        )
    ),
    yaxis=dict(
        title='Calificación en primera oportunidad',
        titlefont=dict(
            size=14,
            color='#000000'
        )
    )
)
f = go.Figure(data = datos, layout = l)
g = py.iplot(f, filename='jupyter-hrs_mcu_v2')
print(g.embed_code)
datos = []
for hrs in orden:
    if hrs != NaN:
        pedazo = d.loc[d['hrsEstudio'] == hrs]
        if pedazo.CF1op.count() > 0:
            datos.append(go.Box(y = d.loc[d['hrsEstudio'] == hrs].CF1op, name = hrs, marker = colores[hrs]))
l = go.Layout(
    title='Encuesta antes del examen de ordinario',
    xaxis=dict(
        title='Horas reportadas de estudio por semana en la segunda mitad',
        titlefont=dict(
            size=14,
            color='#000000'
        )
    ),
    yaxis=dict(
        title='Calificación en primera oportunidad',
        titlefont=dict(
            size=14,
            color='#000000'
        )
    )
)
f = go.Figure(data = datos, layout = l)
g = py.iplot(f, filename='jupyter-hrs_ord_v2')
print(g.embed_code)

['3-5 h' '1-2 h' '2-3 h' '> 5 h' nan '< 1 h']
[nan '3-5 h' '< 1 h' '1-2 h' '> 5 h' '2-3 h' 'Nada']
[nan '2-3 h' '1-2 h' '> 5 h' '3-5 h' '< 1 h' 'Nada']
<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="https://plot.ly/~satuelisa/20.embed" height="525px" width="100%"></iframe>
<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="https://plot.ly/~satuelisa/22.embed" height="525px" width="100%"></iframe>
<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="https://plot.ly/~satuelisa/24.embed" height="525px" width="100%"></iframe>


<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="https://plot.ly/~satuelisa/20.embed" height="525px" width="100%"></iframe>
<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="https://plot.ly/~satuelisa/22.embed" height="525px" width="100%"></iframe>
<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="https://plot.ly/~satuelisa/24.embed" height="525px" width="100%"></iframe>

Las figuras __[8](https://elisa.dyndns-web.com/teaching/comp/datasci/p4.html#f8)__, __[9](https://elisa.dyndns-web.com/teaching/comp/datasci/p4.html#f9)__ y  __[10](https://elisa.dyndns-web.com/teaching/comp/datasci/p4.html#f10)__ son mucho más fáciles de comparar e interpretar.

Al inicio nadie dice que no va a estudiar. Luego resulta que algunos no estudiaron y les fue peor en promedio a los que dicen que sí estudiaron. En medio curso parece que con que hayan estudiado *algo*, ya salen un poco mejor, mientras en el ordinario pinta que estudiando más de una hora a la semana se mejora aún más.

Ibamos a ver también un scatterplot de las horas trabajadas a la semana (cuantificado a nuestros niveles discretos que representaban las opciones) ya que daba una correlación negativa leve. 

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go
import pandas as pd
import ssl

if getattr(ssl, '_create_unverified_context', None):
    ssl._create_default_https_context = ssl._create_unverified_context
d = pd.read_csv("https://elisa.dyndns-web.com/teaching/comp/datasci/graficar.csv")
datos = [go.Scatter(x = d.hrsNum, y = d.CF1op, mode = 'markers')]
g = py.iplot(datos, filename='hrs_de_trabajo')
print(g.embed_code)

<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="https://plot.ly/~satuelisa/26.embed" height="525px" width="100%"></iframe>

Según la figura __[11](https://elisa.dyndns-web.com/teaching/comp/datasci/p4.html#f11)__ no podemos concluir nada.
Esto quedará por resolver con las pruebas estadísticas. 

Lo que quedó por graficar, para esta práctica aún (vamos a seguir dibujando más adelante), era lo de temas por reforzar y medios de apoyo que les interesaban, en términos de cuántos fueron.

Vamos a graduar a gráficas de violin ya que usábamos diagramas de caja-bigote tanto en las figuras anteriores.

In [ ]:
>>> d.temas.unique()
array([nan, 'Unos pocos.', 'Bastantes.',
       'Creo tener que estudiarlos todos luego.',
       'Solo continuar ampliando ', 'hacer mas facil el examen', 'Ninguno.'], dtype=object)
>>> d.temas.value_counts()
Unos pocos.                                146
Bastantes.                                  93
Creo tener que estudiarlos todos luego.     28
Ninguno.                                     3
Solo continuar ampliando                     1
hacer mas facil el examen                    1
Name: temas, dtype: int64

Conviene concentrarnos en los tres niveles que tienen muchas respuestas: Pocos / Bastantes / Todos

In [18]:
import plotly.plotly as py
import plotly.graph_objs as go
import pandas as pd
import ssl

if getattr(ssl, '_create_unverified_context', None):
    ssl._create_default_https_context = ssl._create_unverified_context
d = pd.read_csv("https://elisa.dyndns-web.com/teaching/comp/datasci/graficar.csv")
f = {"data": [{"type": 'violin', "y": d.loc[d['temas'] == "Unos pocos."].CF1op, \
                 "box": {"visible": True}, "line": {"color": 'black'}, "meanline": {"visible": True}, \
                 "fillcolor": '#8dd3c7',"opacity": 0.6,"x0": 'Pocos'}, \
                {"type": 'violin', "y": d.loc[d['temas'] == "Bastantes."].CF1op, \
                 "box": {"visible": True}, "line": {"color": 'black'}, "meanline": {"visible": True}, \
                 "fillcolor": '#d38dc7',"opacity": 0.6,"x0": 'Bastantes'}, \
                {"type": 'violin', "y": d.loc[d['temas'] == "Creo tener que estudiarlos todos luego."].CF1op, \
                 "box": {"visible": True}, "line": {"color": 'black'}, "meanline": {"visible": True}, \
                 "fillcolor": '#d3c78d',"opacity": 0.6,"x0": 'Todos'}],
    "layout" : {
        "title": "Cuántas temas de la unidad de aprendizaje tendrá que reforzar en el futuro",
        "showlegend": False,
        "yaxis": {
            "title": "Calificación en primera oportunidad",
            "zeroline": False,
        }
    }
}
g = py.iplot(f, filename = 'temas_pendientes', validate = False)
print(g.embed_code)

<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="https://plot.ly/~satuelisa/28.embed" height="525px" width="100%"></iframe>


<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="https://plot.ly/~satuelisa/28.embed" height="525px" width="100%"></iframe>

No se nota gran diferencia entre esos tres casos en la figura __[12](https://elisa.dyndns-web.com/teaching/comp/datasci/p4.html#f12)__. 

Ahora lo de cuántas temas en general sienten que les falta reforzar con apoyo de parte de la facultad.
Vamos nuevamente con una gráfica de dispersión. 

Podemos de hecho poner en una misma figura lo de temas por reforzar y cuántas formas de apoyo y cuántos medios indicaron que les interesaría usar. 

Para que no se sobrepongan tanto, apliquemos un pequeño desplazamiento horizontal a dos de los tres conjuntos.

In [22]:
import plotly.plotly as py
import plotly.graph_objs as go
import pandas as pd
import ssl

if getattr(ssl, '_create_unverified_context', None):
    ssl._create_default_https_context = ssl._create_unverified_context
d = pd.read_csv("https://elisa.dyndns-web.com/teaching/comp/datasci/graficar.csv")
m1 = {'size': 5, 'color': 'rgba(170, 0, 0, .7)'}
m2 = {'size': 6, 'color': 'rgba(0, 170, 0, .6)'}
m3 = {'size': 7, 'color': 'rgba(0, 0, 170, .5)'}
delta = 0.2
datos = [go.Scatter(x = d.cuantasTemas - delta, y = d.CF1op, mode = 'markers', marker = m1, name="Temas"),\
         go.Scatter(x = d.formasApoyo, y = d.CF1op, mode = 'markers', marker = m2, name="Apoyos"), \
         go.Scatter(x = d.cuantosMedios + delta, y = d.CF1op, mode = 'markers', marker = m3, name="Medios")]
f = {"data": datos, 
    "layout" : {
        "title": "Interés en apoyo en temas generales",
        "showlegend": True,
        "xaxis": {
            "title": "Número de opciones seleccionadas",
            "zeroline": False,
        },
        "yaxis": {
            "title": "Calificación en primera oportunidad",
            "zeroline": False,
        }
    }
}
g = py.iplot(f, filename='apoyo')
print(g.embed_code)

<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="https://plot.ly/~satuelisa/30.embed" height="525px" width="100%"></iframe>


<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="https://plot.ly/~satuelisa/30.embed" height="525px" width="100%"></iframe>

El resultado es la figura __[13](https://elisa.dyndns-web.com/teaching/comp/datasci/p4.html#f13)__ que nos indica que gente de todas las imaginables calificaciones dice que cero, mayormente porque ahora hasta los que no contestaron salen como ceros por cómo se codificó esto en la práctica anterior.

Lo único que puedo concluir de la gráfica es que se va cerrando un poco hacía los números grandes: los que eligen seis o más ya desempeñan un poco peor que los que elegieron entre una y cinco opciones. 

Esto se podría probar con una prueba estadística en una práctica futura.

En su reporte, incluyan por lo menos tres diferentes tipos de gráficas e intenten concluir algo sobre sus datos.